In [25]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import sqrt, log

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * sqrt(self.emb_size)


In [35]:
class TokenTypeTransformer(nn.Module):
    def __init__(self,
                 num_layers: int,
                 emb_size: int,
                 nhead: int,
                 vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(TokenTypeTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size,
                                                nhead=nhead,
                                                dim_feedforward=dim_feedforward,
                                                dropout=dropout,
                                                batch_first=True
                                                )
        self.transformer: TransformerEncoder = TransformerEncoder(encoder_layer=encoder_layer,
                                              num_layers=num_layers)
        self.generator = nn.Linear(emb_size, vocab_size)
        self.tok_emb = TokenEmbedding(vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
    

    def forward(self,
                src: Tensor,
                src_mask: Tensor = None,
                src_padding_mask: Tensor = None,
                is_causal: bool = None):
        src_emb = self.positional_encoding(self.tok_emb(src))
        outs = self.transformer(src=src_emb, mask=src_mask, src_key_padding_mask=src_padding_mask, is_causal=is_causal)
        return self.generator(outs)

In [31]:
from dataset import TokenTypesDataset


train_dataset = TokenTypesDataset(folder="../rnn_tokentype_data/train")
val_dataset = TokenTypesDataset(folder="../rnn_tokentype_data/validation", train=False, vocabs=(train_dataset.token2idx, train_dataset.idx2token), max_length=train_dataset.max_length)
test_dataset = TokenTypesDataset(folder="../rnn_tokentype_data/test", train=False, vocabs=(train_dataset.token2idx, train_dataset.idx2token), max_length=train_dataset.max_length)
assert val_dataset.vocab_size == train_dataset.vocab_size == test_dataset.vocab_size
assert val_dataset.max_length == train_dataset.max_length == test_dataset.max_length


In [41]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = TokenTypeTransformer(
                            num_layers=4,
                            emb_size=512,
                            nhead=8,
                            vocab_size=train_dataset.vocab_size
                            )

test_src = torch.Tensor([[1, 2, 3, 4]])
model(test_src).shape

torch.Size([1, 4, 33])

In [45]:
import torch.nn as nn
from torch.utils.data import DataLoader
from train import train

NUM_EPOCHS = 30

torch.manual_seed(42)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = None
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.pad_id)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

train(model, optimizer, scheduler, train_loader, test_loader,  NUM_EPOCHS)

Training 1/30:   0%|          | 0/1841 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x114a60550>
Traceback (most recent call last):
  File "/Users/ivanbondyrev/ml-course-hse/2022-fall/homeworks-practice/ml1_2022/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/Users/ivanbondyrev/ml-course-hse/2022-fall/homeworks-practice/ml1_2022/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framewor

AssertionError: only bool and floating types of mask are supported